# División *k-fold*

Dividir un conjunto de datos conjuntos de entrenamiento y prueba, es una buena técnica para cuando queremos evaluar un algoritmo de *machine learning*, pero, dividir los datos en solo dos conjuntos puede arrojar resultados muy sesgados.

*K-fold* es una alternativa para evitar este problema, ya que, nos permite dividir los datos en *k* particiones homogéneas, dónde, se utilizará cada una de estas particiones como conjunto de prueba mientras las demás se utilizan como entrenamiento, en un proceso iterativo de *k* iteraciones.

En la siguiente imagen se ilustra como se dividen los datos, cuando *k = 5*. El color verde se refiere al conjunto de entrenamiento y el color amarillo al conjunto de prueba.

![k-fold](kfold.PNG)

Hagamos el código para realizar las divisiones.

Importamos librerías de utilidad.

In [2]:
import pandas as pd
import numpy as np

Abrimos la base de datos que vamos a utilizar, **EnfCoronaria**.

In [3]:
df = pd.read_csv("EnfCoronaria.csv")
df.head()

,sbp,famhist,obesity,age,chd
0,160,1,25.30,52,1
1,144,0,28.87,63,1
2,118,1,29.14,46,0
3,170,1,31.99,58,1
4,134,1,25.99,49,1


Las particiones las realizaremos a través de una función que llamaremos **particion**, en la cual, haremos las particiones de la manera más homogéna que podamos, esta función recibirá como parámetros la base de datos **df** y el número de particiones a realizar **k**. Creamos una lista donde guardaremos las divisiones, llamada **datos**, obtenemos el número de elementos en la base de datos y lo guardamos en **tamano**, este valor tiene que ser entero, por lo tanto, al momento de guardar el valor, lo convertimos a entero (a esto se le conoce como *casting*), porque la división entre el número de datos y las particiones a crear, usualmente, será un número tipo *float*. El siguiente paso es hacer un ciclo *for* donde iremos asignando los índices de cada partición, el rango de cada partición lo determinamos utilizando la función ***range***, donde le mandaremos los parámetros, **i\*tamano** (límite inferior) y **(i+1)\*tamano** (límite superior), este rango lo vamos a guardar como un arreglo, y lo agregamos a la lista **datos**, este ciclo lo ejecutaremos *k-1* veces, ya que para la última partición, los límites serán **(i+1)\*tamano** (límite inferior) y ***len(df)*** (límite superior), de modo que, si se realizó un *casting* cuando obtuvimos **tamano**, no se pierdan elementos, por ejemplo, si tenemos 27 datos en total y queremos 5 particiones, el tamaño de la partición sería de 27/5 = 5.4, al realizar el *casting* el tamaño nos quedaría de 5, pero, si hacemos todas las particiones de 5 elementos al final tendríamos 25 datos en lugar de 27, para evitar esto, lo que hacemos es que las primeras 4 particiones las hacemos de 5 elementos y, la última partición, la hacemos de los elementos restantes, 7 en este caso, y así al final tenemos los 27 datos, aunque nuestra última partición es ligeramente más grande.

In [4]:
def particion(df, k):
    # lista donde guardaremos las particiones
    datos = []
    # obtenemos el total de datos en la BD
    tamano = int(len(df)/k)
    # comenzamos a realizar las particiones
    for i in range(k-1):
        # indices de cada partición
        division = np.array(range(i*tamano, (i+1)*tamano))
        # guardamos la partición en datos
        datos.append(np.array(division))
    # última división
    datos.append(np.array(range((i+1)*tamano, len(df))))
    return datos

Ya con las particiones creadas, haremos otra función llamada **conjunto**, que nos servirá para obtener los conjuntos de entrenamiento y prueba. Esta función recibe tres parámetros, la base de datos **df**, la lista de particiones **datos**, y el índice **indice** que indica el número de iteración y, por ende, la posición del conjunto de prueba. Para crear un conjunto de datos a partir de un *data frame*, usamos la función ***iloc*** de ***pandas*** que filtra los datos de acuerdo con los índices que recibe como parámetro. El conjunto de prueba, lo podemos crear mandando **datos[indice]**, como parámetro a ***iloc***; el conjunto de entrenamiento, consiste en todos los datos, excepto los que usamos como prueba, entonces para obtener estos datos podemos obtener los índices de todos los datos en **df**, usando ***np.array(range(len(df))***, y eliminando los índices del conjunto de datos con la función ***delete*** de ***numpy***, está función, recibe como parámetros el *array* sobre el que se va a trabajar y las posiciones que se van a eliminar, entonces, usando ***np.delete(ind, datos[indice])***, le estamos diciendo que trabaje sobre los índices de todo el archivo y después elimine los que usamos como prueba. Hecho esto, usamos de nuevo ***iloc*** para obtener los datos correspondientes al conjunto de entrenamiento.

In [5]:
def conjunto(df, datos, indice):
    # creamos el conjunto de prueba
    prueba = df.iloc[datos[indice]]
    # sacamos los indices de toda la BD
    ind = np.array(range(len(df)))
    # eliminamos los indices del conjunto de prueba
    ind_ent = np.delete(ind, datos[indice])
    # creamos el conjunto de entrenamiento
    entrenamiento = df.iloc[ind_ent]
    return entrenamiento, prueba

Probemos nuestro algoritmo.

In [6]:
# numero de particiones que queremos
k = 4
# mandamos llamar a division() para crear las particiones
datos = particion(df, k)
datos

[array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114]),
 array([115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
        128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
        141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
        167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 1

In [7]:
# creamos dos listas donde guardaremos nuestros datos
train = []
test = []
# obtenemos los conjuntos llamando a conjunto()
for i in range(k):
    entrenamiento, prueba = conjunto(df, datos, i)
    print("Iteración:", i + 1)
    print("Entrenamiento:")
    print(entrenamiento)
    print("Prueba:")
    print(prueba)
    # agregamos los conjuntos a las listas
    train.append(entrenamiento)
    test.append(prueba)

Iteración: 1
Entrenamiento:
     sbp  famhist  obesity  age  chd
115  128        1    28.41   48    0
116  140        0    21.91   32    1
117  154        0    20.60   42    0
118  150        1    23.35   61    1
119  130        0    28.02   27    0
..   ...      ...      ...  ...  ...
457  214        0    28.45   58    0
458  182        0    28.61   52    1
459  108        0    20.09   55    0
460  118        0    27.35   40    0
461  132        1    14.70   46    1

[347 rows x 5 columns]
Prueba:
     sbp  famhist  obesity  age  chd
0    160        1    25.30   52    1
1    144        0    28.87   63    1
2    118        1    29.14   46    0
3    170        1    31.99   58    1
4    134        1    25.99   49    1
..   ...      ...      ...  ...  ...
110  114        0    25.51   16    0
111  168        1    26.18   54    1
112  134        0    21.03   37    0
113  174        1    25.27   61    1
114  116        0    19.40   59    1

[115 rows x 5 columns]
Iteración: 2
Entrenamiento:


Y preguntamos si queremos exportar los datos a un archivo. Si la respuesta es afirmativa, guardaremos cada subconjunto de entrenamiento con su respectivo subconjunto de prueba, enumerándolos desde 0 hasta *k*-1.

In [8]:
guardar = input("¿Desea guardar los datos en un archivo? s/n\n")
if guardar.lower() == "s":
    for i in range(k):
        n_train = "train_" + str(i) + ".csv"
        train[i].to_csv(n_train)
        n_test = "test_" + str(i) + ".csv"
        test[i].to_csv(n_test)

¿Desea guardar los datos en un archivo? s/n
n


Hecho esto, los archivos se crearán en el directorio en el que estemos trabajando.
![archivos](archivos_kfold.png)